In [1]:
import numpy as np
from scipy.linalg import svd
import pandas as pd

# To find pgm files
import glob

# To read and show pgm files
from skimage.io import imread, imshow

import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
%precision %.5f
plt.rc('text', usetex=True)
plt.rc('font',**{'family':'serif','serif':['Palatino']})

In [2]:
def soft_thresholding(mat, epsilon):
    """
    Soft thresholding operator
    Proximal operator of the L1 norm
    :param mat: matrix to threshold
    :param epsilon: value of threshold
    :return mat: thresholded matrix
    """
    mat[np.abs(mat) < epsilon] = 0
    mat[mat > epsilon] -= epsilon
    mat[mat < -epsilon] += epsilon

    return mat

In [3]:
def svalue_thresholding(mat, epsilon):
    """
    Singular value thresholding
    Proximal operator of the nuclear norm
    :param mat: matrix of svalue to threshold
    :param epsilon: value of threshold
    :return mat: thresholded matrix
    """
    u, s, v = svd(mat, full_matrices = False, lapack_driver = 'gesvd')
    soft_thresholding(s, epsilon)
    return u.dot(np.diag(s).dot(v))

In [4]:
def lrmc(X, W, tau, beta, tol = 10e-3, A_init = 0, n_iter = 1000):
    """
    Finds a low-rank matrix A whose entries in W coincide
    with those of X by using the SVT algorithm.
    :param X: DxN data matrix
    :param W: DxN binary matrix denoting known (1) or missing (0) entries
    :param tau: Parameter of the optimization problem
    :param beta: Step size of the dual gradient ascent step
    :param tol: tolerance for convergence boost speed
    :return A: Low-rank completion of the mtrix X
    """
    A = A_init
    Z = beta * (X * W - A * W)
    stop = False
    while (stop == False and n_iter):
        n_iter -= 1
        Z_prev = Z
        A = svalue_thresholding(Z * W, tau)
        Z = Z + beta * (X - A) * W
        if (np.linalg.norm(Z - Z_prev) / (tol + np.linalg.norm(Z_prev)) <= tol):
            stop = True
#         print (np.linalg.norm(Z - Z_prev) / (tol + np.linalg.norm(Z_prev)))
    return A

In [11]:
def get_train_set(mat, percentage):
    """
    Given a percentage of training and the data represented by mat,
    return the index of training set and test set
    """
    # Percentage of training data set length
    sets = np.arange(len(mat))
    np.random.shuffle(sets)
    return sets[:int(percentage *len(mat) / 100.)], sets[int(percentage * len(mat) / 100):]

# Data moovielens

In [5]:
df = pd.read_csv("ratings_medium_n4_Horror_Romance_42.csv")
df

,userId,genreId,movieId,rating,timestamp,movieInd,userInd
0,4,1,610,4.0,949982238,23,0
1,4,1,1200,5.0,949896244,41,0
2,4,1,1214,5.0,949810261,42,0
3,4,1,1219,5.0,949779173,44,0
4,4,1,1258,5.0,949918743,47,0
5,4,1,1332,4.0,949896275,60,0
6,4,1,1334,5.0,949982274,62,0
7,4,1,1387,5.0,949810261,76,0
8,4,1,1388,4.0,949810302,77,0
9,4,1,1994,5.0,949896309,121,0


## Horror set

In [9]:
df_horror = df[df['genreId'] == 1]
df_horror

,userId,genreId,movieId,rating,timestamp,movieInd,userInd
0,4,1,610,4.0,949982238,23,0
1,4,1,1200,5.0,949896244,41,0
2,4,1,1214,5.0,949810261,42,0
3,4,1,1219,5.0,949779173,44,0
4,4,1,1258,5.0,949918743,47,0
5,4,1,1332,4.0,949896275,60,0
6,4,1,1334,5.0,949982274,62,0
7,4,1,1387,5.0,949810261,76,0
8,4,1,1388,4.0,949810302,77,0
9,4,1,1994,5.0,949896309,121,0


In [32]:
horror_train, horror_test = get_train_set(df_horror, 10)
MH = df_horror.as_matrix()
MH_train = MH.copy()
MH_train[horror_train, 3] = 0
WH_train = np.ones((MH.shape))
WH_train[horror_train, 3] = 0

In [35]:
[((lrmc(MH_train, WH_train, tau = t, beta = 2)[:, 3] - MH[:, 3]) ** 2).sum() / 5600 for t in np.power(10, np.arange(20))]

[11.71363,
 9.60921,
 0.59089,
 0.31269,
 0.96827,
 0.15575,
 8.54889,
 8.65031,
 8.64761,
 8.50132,
 7.30513,
 1.77220,
 4.08013,
 1.60392,
 11.96116,
 11.96116,
 11.96116,
 11.96116,
 11.96116,
 50887735121484518264184478652567843241984.00000]

## Romance set

In [10]:
df_romance = df[df['genreId'] == 2]
df_romance

,userId,genreId,movieId,rating,timestamp,movieInd,userInd
18,4,2,356,5.0,949919763,862,0
19,4,2,357,5.0,949919681,863,0
20,4,2,380,3.0,949810534,867,0
21,4,2,440,4.0,949919802,870,0
22,4,2,903,5.0,949919189,908,0
23,4,2,1022,5.0,949949638,921,0
24,4,2,1257,5.0,949811602,935,0
25,4,2,1265,5.0,949919591,937,0
26,4,2,1674,5.0,949896244,968,0
27,4,2,1917,4.0,949810688,990,0


In [37]:
romance_train, romance_test = get_train_set(df_romance, 10)
MR = df_romance.as_matrix()
MR_train = MR.copy()
MR_train[romance_train, 3] = 0
WR_train = np.ones((MR.shape))
WR_train[romance_train, 3] = 0

In [39]:
[((lrmc(MR_train, WR_train, tau = t, beta = 2)[:, 3] - MR[:, 3]) ** 2).sum() / 5600 for t in np.power(10, np.arange(20))]

[18.32936,
 15.58657,
 1.43302,
 0.17651,
 0.70392,
 1.52306,
 13.23205,
 13.42132,
 13.52928,
 13.32264,
 11.56456,
 2.15858,
 17.94729,
 13.25094,
 18.64799,
 18.64799,
 18.64799,
 18.64799,
 18.64799,
 50883085333226719327268455921061258592256.00000]

## Horror + romance set

In [40]:
all_train, all_test = get_train_set(df, 10)
M = df.as_matrix()
M_train = M.copy()
M_train[all_train, 3] = 0
W_train = np.ones((M.shape))
W_train[all_train, 3] = 0

In [41]:
[((lrmc(M_train, W_train, tau = t, beta = 2)[:, 3] - M[:, 3]) ** 2).sum() / 5600 for t in np.power(10, np.arange(20))]

[30.20301,
 26.68011,
 5.18353,
 0.44720,
 1.63204,
 2.27877,
 21.63066,
 22.16454,
 22.10010,
 22.06324,
 19.80992,
 4.89135,
 11.68781,
 10.94788,
 30.60915,
 30.60915,
 30.60915,
 30.60915,
 30.60915,
 50885744359757405813837501695405645103104.00000]

In [48]:
[((lrmc(M_train, W_train, tau = t, beta = 2)[:, 3] - M[:, 3]) ** 2).sum() / 5600 for t in [1160, 1170, 1180]]

[0.34880, 0.34943, 0.35056]